In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
import pyreader
import numpy as np
import csv
from batchmake import Batcher
import pprint

In [2]:
datas= 'data_samples/'
vocabulary = 'vocab.csv'
vocabs = []
with open(vocabulary, 'r', newline='', encoding='utf-8') as vocab:
    words = csv.reader(vocab)
    for word in words:
        vocabs.append(word)
# word_to_id='data_samples/mapping.map'
# with open(word_to_id, 'rb') as f:
#     word_to_id = pickle.load(f)
# inv_map = {v:k for k, v in word_to_id.items() }
# batch_data = datas+'output.txt.part0'
batch_data = datas+'preprocess.part0'
with open(batch_data, 'rb') as f:
    picked_data = pickle.load(f)
print(len(vocabs))

22669


In [3]:
def attention(inputs, attention_size):
    inputs_shape = inputs.shape
    sequence_length = inputs_shape[1].value
    hidden_size = inputs_shape[2].value
    
    w_omega = tf.Variable(tf.random_normal([hidden_size,attention_size], stddev=0.1))
    b_omega = tf.Variable(tf.random_normal([attention_size], stddev=0.1))
    u_omega = tf.Variable(tf.random_normal([attention_size], stddev=0.1))
    
    # batch *k , attention_size
    v = tf.tanh(tf.matmul(tf.reshape(inputs, [-1,hidden_size]), w_omega)+
               tf.reshape(b_omega,[1,-1]))
    
    vu = tf.matmul(v, tf.reshape(u_omega,[-1,1])) # batch*k,1
    exps = tf.reshape(tf.exp(vu), [-1, sequence_length])  # (batch, k)
    alphas = exps / tf.reshape(tf.reduce_sum(exps,1), [-1,1])
    #(batch,k)
    output = tf.reduce_sum(inputs * tf.reshape(alphas, [-1, sequence_length,1]),1)
    # batch * sequence * hidden -> batch * hidden
    return output

In [4]:
hidden_size=32
sequence=5
embedding_dim=100
attention_size = 50
batch_size=1
vocab_size=len(vocabs)
grucell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
X = tf.placeholder(tf.int32, [None, sequence], name='inputs_xdata')
Y = tf.placeholder(tf.float32, [None, sequence], name='targets_ydata')
seq_len = tf.placeholder(tf.int32, [None], name='seq_len')

embedding_variable = tf.Variable(tf.random_uniform([vocab_size, embedding_dim],-1.0,1.0), trainable=True)
batch_embedded = tf.nn.embedding_lookup(embedding_variable, X)


with tf.variable_scope('RNN'):
    outputs, states = tf.nn.dynamic_rnn(grucell,
                                        inputs=batch_embedded,
                                        sequence_length=seq_len,
                                        dtype=tf.float32)

In [5]:
attention_output = attention(outputs, attention_size=attention_size)

In [6]:
keep_prob = 0.5
drop = tf.nn.dropout(attention_output, keep_prob=keep_prob)
w = tf.Variable(tf.truncated_normal([hidden_size,1], stddev=0.1))
b = tf.Variable(tf.constant(0., shape=[1]))
y_hat = tf.nn.xw_plus_b(drop,w,b)

y_hat = tf.squeeze(y_hat)

loss = -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_hat, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss)

In [8]:
file_name_queue = tf.train.string_input_producer(['inputs'])
reader = tf.TFRecordReader()
_, single_x = reader.read(file_name_queue)

context_features = {
    "length": tf.FixedLenFeature([], dtype=tf.int64)
}
sequence_features = {
    "tokens": tf.FixedLenSequenceFeature([], dtype=tf.int64),
    "labels": tf.FixedLenSequenceFeature([], dtype=tf.int64)
}


context_parsed, sequence_parsed = tf.parse_single_sequence_example(serialized=single_x,
                                 context_features=context_features,
                                 sequence_features=sequence_features
                                )
# batch_seq, batch_x, batch_y = tf.train.shuffle_batch(tensors=[context_parsed['length'],sequence_parsed['tokens'],sequence_parsed['labels']],
#                                                      batch_size=batch_size, 
#                                                      capacity=5000,
#                                                      num_threads=1,
#                                                      min_after_dequeue=1000,
#                                                     )
batch_seq, batch_x, batch_y = tf.train.batch(tensors=[context_parsed['length'],sequence_parsed['tokens'],sequence_parsed['labels']],
                                                     batch_size=batch_size, 
                                                     capacity=5000,
                                                     num_threads=1,
                                             dynamic_pad=True
                                                    )


Ses = tf.Session()

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=Ses, coord=coord)
Ses.run(tf.global_variables_initializer())

# print(tokens_['tokens'])
for i in range(1000):
    seq_, x_in, y_in = Ses.run([batch_seq, batch_x, batch_y])
    feed_data = {X:x_in, Y:y_in, seq_len:seq_}
#     print(feed_data)
#     try:
    lo, opt = Ses.run([loss,optimizer], feed_dict=feed_data)
#     print(feed_data)
#     except:
#         print('error occured!!!  ',i, feed_data)
#         break
#     print('loss',lo)

KeyboardInterrupt: 

In [11]:
filename_queue = tf.train.string_input_producer(['./inputs.csv'])
reader = tf.TextLineReader()
key, record_string = reader.read_up_to(filename_queue,num_records=sequence)
# key, record_string = reader.read(filename_queue)
seq_lengths = [[11]]
# record_string = tf.reshape(record_string, [-1])
record_defaults = [[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1]]
batc_record = tf.cast(tf.decode_csv(record_string, record_defaults=record_defaults),tf.int32)
batc_record = tf.reshape(batc_record,[input_dim,sequence])
# target_string = tf.slice(record_string, [1,0],[1,-1])
# file_content = tf.read_file(filename_queue)
# batc_record = tf.transpose(batc_record,[0,2,1])
# output = tf.train.batch([batc_record], batch_size=batch_size, dynamic_pad=True,allow_smaller_final_batch=False)
output = tf.train.shuffle_batch(tensors=[batc_record],
                                batch_size=batch_size,
                                capacity = 5000,
                                num_threads=1,
                                min_after_dequeue=1000
                               )
output = tf.transpose(output,[0,2,1])
target = tf.slice(output, [0,0,1],[-1,-1,-1])
print(output,target)
# output = tf.reshape(output,[2,3,11])
Ses = tf.Session()

# sequences, output = tf.contrib.training.bucket_by_sequence_length(input_length=seq_lengths,
#                                              tensors=[batc_record],
#                                              batch_size=1,
#                                              bucket_boundaries=[11,13],
#                                              dynamic_pad=True
#                                              )

Tensor("transpose_1:0", shape=(1, 5, 11), dtype=int32) Tensor("Slice_1:0", shape=(1, 5, 10), dtype=int32)


In [12]:
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=Ses, coord=coord)
Ses.run(tf.global_variables_initializer())

for i in range(1000):
    output_,target_ = Ses.run([output,target])
    feed_data = {X:output_, Y:target_}
#     try:
    lo, opt = Ses.run([loss,optimizer], feed_dict=feed_data)
#     print(feed_data)
#     except:
#         print('error occured!!!  ',i, feed_data)
#         break
    print('loss',lo)
# batch_x_ = Ses.run([batch_x])

# print(np.array(batch_x_[0]))
# print(np.array(batch_x_[0]))
# print(batch_x.shape)
# output_,target_=Ses.run([output,target])
# print(np.array(target_))
# print(np.array(target_).shape)

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, Input to reshape is a tensor with 11 values, but the requested shape has 55
	 [[Node: Reshape_6 = Reshape[T=DT_INT32, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](Cast/x, Reshape_6/shape)]]
loss -36.6007
loss 41.6747
loss 23.0561
loss -17.2172
loss 101.85
loss 65.1391
loss 12.8645
loss -30.7384
loss 86.8514
loss 31.4633
loss -3.43664
loss -3.66925
loss 53.6685
loss -1.15071
loss -7.68049
loss 65.4639
loss -6.19446
loss -2.29135
loss -0.589737
loss -38.9972
loss 32.6997
loss -18.9629
loss 2.42858
loss -3.93419
loss 115.48
loss -14.4266
loss 19.9054
loss -106.323
loss -7.46684
loss -32.4471
loss -73.1171
loss -24.6179
loss -164.784
loss 2.44641
loss 41.001
loss -33.9733
loss -130.663
loss 51.621
loss -631.38
loss -10.008
loss -7.0104
loss -7.89129
loss -39.6123
loss -38.734
loss 98.0991
loss -3.1538
loss -71.763
loss -198.999
loss 1.77701
loss -4.887

OutOfRangeError: RandomShuffleQueue '_7_shuffle_batch_1/random_shuffle_queue' is closed and has insufficient elements (requested 1, current size 0)
	 [[Node: shuffle_batch_1 = QueueDequeueManyV2[component_types=[DT_INT32], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](shuffle_batch_1/random_shuffle_queue, shuffle_batch_1/n)]]

Caused by op 'shuffle_batch_1', defined at:
  File "D:\python3_5anaconda\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\python3_5anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\python3_5anaconda\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "D:\python3_5anaconda\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\python3_5anaconda\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "D:\python3_5anaconda\lib\site

In [6]:
keep_prob = 0.5
drop = tf.nn.dropout(attention_output, keep_prob=keep_prob)

w = tf.Variable(tf.truncated_normal([hidden_size,1], stddev=0.1))
b = tf.Variable(tf.constant(0., shape=[1]))
y_hat = tf.nn.xw_plus_b(drop,w,b)
y_hat = tf.squeeze(y_hat)

loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_hat, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss)

Tensor("Sum_1:0", shape=(?, 32), dtype=float32)


In [7]:
s = tf.Session()

In [8]:
batcher = Batcher([batch_data], batch_size=batch_size, seq_length=sequence) #batchsize, sequence

for batch in batcher:
    for i, feed_data  in enumerate(batcher.sequence_iterator(batch)):
        inputs,targets,masks,identifiers,actual_lengths = feed_data
        print(inputs.shape , targets.shape, masks.shape, identifiers.shape,actual_lengths.shape)
        print(actual_lengths)
        
        feed_data = {X:inputs, Y:targets}
        #inputs :  batch, sequence
        
        lo, opt = s.run([loss,optimizer], feed_dict=feed_data)
        
            
    break

Loaded data partition data_samples/preprocess.part0 with 481 examples
[<pyreader.Container object at 0x000001C9A750A518>, <pyreader.Container object at 0x000001C9A78A1898>, <pyreader.Container object at 0x000001C9A79122B0>, <pyreader.Container object at 0x000001C9A79D2BA8>, <pyreader.Container object at 0x000001C9A7953358>, <pyreader.Container object at 0x000001C9A79803C8>, <pyreader.Container object at 0x000001C9A79EADD8>, <pyreader.Container object at 0x000001C9A7A06978>, <pyreader.Container object at 0x000001C9A7A12278>, <pyreader.Container object at 0x000001C9A7A12390>, <pyreader.Container object at 0x000001C9A7A124A8>, <pyreader.Container object at 0x000001C9A7A125C0>, <pyreader.Container object at 0x000001C9A7A126D8>, <pyreader.Container object at 0x000001C9A7A127F0>, <pyreader.Container object at 0x000001C9A7A12908>, <pyreader.Container object at 0x000001C9A7A12A20>]
(5, 16) (5, 16) (5, 16, 1) (16, 5) (16,)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


ValueError: Cannot feed value of shape (5, 16) for Tensor 'Placeholder:0', which has shape '(?, 5, 1)'